In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/LENOVO/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
from datasets import DatasetDict
train_val_split = dataset["train"].train_test_split(test_size=0.2)
split_dataset = DatasetDict({
    'train':train_val_split['train'],
    'validation': train_val_split['test'],
    'test':dataset['test']
})
split_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 520000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 130000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [4]:
split_dataset.set_format(type="pandas")
def int_2_str(row):
    return split_dataset['train'].features['label'].int2str(row)
df = split_dataset['train'][:]
df["label_name"] = df["label"].apply(int_2_str)
df.head()

,label,text,label_name
0,3,"Good food, very good actually- the oysters tas...",4 stars
1,0,Don't be fooled by their prices. These people...,1 star
2,1,This restaurant is only appropriate for a quic...,2 star
3,2,The marinara was good. The prices were bad and...,3 stars
4,0,"Poor Quality, rude employees, My recommendatio...",1 star


In [5]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [6]:
def tokenize(batch):
    return tokenizer(batch['text'],padding = True,truncation=True)

In [7]:
split_dataset.reset_format()
dataset_encoded = split_dataset.map(tokenize,batched = True,batch_size = True)

Map:   0%|          | 0/520000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
print(dataset_encoded.column_names)

{'train': ['label', 'text', 'input_ids', 'attention_mask'], 'validation': ['label', 'text', 'input_ids', 'attention_mask'], 'test': ['label', 'text', 'input_ids', 'attention_mask']}


In [9]:
from transformers import AutoModelForSequenceClassification
import torch
num_labels = 5
device = "cuda" if torch.cuda.is_available() else "cpu"

model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

D:\Jupyter\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\Jupyter\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [11]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
from transformers import Trainer, TrainingArguments

batch_size =16
logging_steps = len(dataset_encoded["train"])
model_name = f"{model_ckpt}-finetuned-rating-stars"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=0.001,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size= 16,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True, 
                                  log_level="error")

In [19]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
  
                  tokenizer=tokenizer)
trainer.train();

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 